In [2]:
import time
import json
import re
import html
import math
import os

# Scraper Settings

In [3]:
userLogin = ''
userPwd = ''

In [4]:
Interval = 5 #time between switching the page
profileId = 'ACwAAA5x2nYB-HnFP_cCjbggW53mECNDfDzYHKA'
# profileId = sys.argv[1] #profileID as input parameter

searched_json = 'https://www.linkedin.com/sales-api/salesApiLeadSearch?q=searchQuery' #searched object name saved in requests


# Functions

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import DesiredCapabilities #https://gist.github.com/lorey/079c5e178c9c9d3c30ad87df7f70491d
import json
import random

def StartDriver():
    #browser settings
    caps = webdriver.DesiredCapabilities.CHROME.copy()
    options = webdriver.ChromeOptions()
    options.add_argument("disable-gpu")

    #options.add_argument("headless")
    options.add_experimental_option("prefs", {
    #"download.default_directory": r"/Volumes/GoogleDrive/My Drive/Notion/",
    "download.Prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
    })
    options.add_argument("no-default-browser-check")
    options.add_argument("no-first-run")
    options.add_argument("no-sandbox")      

    # make chrome log requests
    capabilities = DesiredCapabilities.CHROME
    capabilities['goog:loggingPrefs'] = {'performance': 'ALL'}

    driver = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=capabilities, options=options)
    driver.maximize_window()
    
    global logs_raw
    global logs
    logs_raw = driver.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]
    return(driver)

In [6]:
def login(driver, login, pwd):    
    start_page = 'https://www.linkedin.com/login'

    #Open LinkedIn Login Page
    driver.get(start_page)
    print('Opened page: ' + driver.title)
    time.sleep(1)
    email_input = driver.find_element(by=By.XPATH, value='//*[@id="username"]')
    time.sleep(1)
    email_input.send_keys(login)
    time.sleep(2)
    pass_input = driver.find_element(by=By.XPATH, value='//*[@id="password"]')
    pass_input.send_keys(pwd)
    pass_input.send_keys(Keys.ENTER)
    time.sleep(5)    

In [7]:
def GetJson(driver, url, searched_json):
    # 3 arguments: active driver, url to scrape, search json
    driver.get(connectionPageUrl)
    time.sleep(Interval)    

    # Filter Json logs
    def log_filter(log_):
        return (
            # is an actual response
            log_["method"] == "Network.responseReceived"
            # and json type only
            and "json" in log_["params"]["response"]["mimeType"]
        )               

    logs_raw = driver.get_log("performance") #get all responses
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw] #get message for each response
    
    # iterate for all json responses from logs
    for log in filter(log_filter, logs):
        #api response name
        response_name = json.dumps(log["params"]["response"]["url"])
#         print(response_name)
        
        #if search_json name is equal to current json response name then get the response body
        if searched_json in response_name:
            request_id = log["params"]["requestId"]
            json_code = (driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id}))['body']
            json_code_loaded = json.loads(json_code) # załaduj jsona 
            json_code_parsed = json.dumps(json_code_loaded) # dump jsona aby pozbyć się pojedynczyć "'"
            response = html.unescape(json_code_parsed) # escape znaków htmlowych typu &amp;           
            
            
            return(response)

# Scraping part

In [12]:
# Run the Chrome Browser
driver = StartDriver()

[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - Driver [/home/budmanz/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache


In [8]:
# Login to Linkedin.com
login(driver, userLogin, userPwd)

Opened page: LinkedIn Login, Sign in | LinkedIn


In [9]:
pageNumber = 1 #start from page 1
connectionPageRootUrl = 'https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(doLogHistory%3Atrue)%2Cfilters%3AList((type%3ACONNECTION_OF%2Cvalues%3AList((id%3A' + profileId +  '%2CselectionType%3AINCLUDED)))))&page='
connectionPageUrl = connectionPageRootUrl + str(pageNumber)
print(connectionPageUrl)

https://www.linkedin.com/sales/search/people?query=(recentSearchParam%3A(doLogHistory%3Atrue)%2Cfilters%3AList((type%3ACONNECTION_OF%2Cvalues%3AList((id%3AACwAAA5x2nYB-HnFP_cCjbggW53mECNDfDzYHKA%2CselectionType%3AINCLUDED)))))&page=1


In [10]:
# for each page get the json saved in requests
try:
    json_code = GetJson(driver, connectionPageUrl, searched_json)
    file_name = str(profileId) + '__page' + str(pageNumber) + '.json'
#     print(json_code)
    with open(file_name, 'w') as outfile:
        outfile.write(json_code)
    
    
    totalResults = re.sub('\D', '', json.loads(json_code)['metadata']['totalDisplayCount'])
    numOfPages = math.ceil(int(totalResults)/25) #dividate number of connection per 25 connection per page

    #pagination
    while pageNumber < numOfPages:
        pageNumber += 1
        connectionPageUrl = connectionPageRootUrl + str(pageNumber) 

        #Continue looping when error occur for a page
        try:
            json_code = GetJson(driver, connectionPageUrl, searched_json)
            file_name = str(profileId) + '__page' + str(pageNumber) + '.json'
#             print(json_code)
            with open(file_name, 'w') as outfile:
                outfile.write(json_code)        
        except Exception as error:
            print(error)
        
except Exception as error:
    print(error)


write() argument must be str, not None


# Merging jsons

In [11]:
import os
final_data = {}
for file in os.listdir('/Users/michalpietrawski/Library/CloudStorage/OneDrive-Personal/GitHub/python-scripts/jupiter_notebook/Noble/'):
    if file.startswith(profileId) and file.endswith('.json'):
        try:
            with open(file) as f:
#                 print(file)
                data = json.load(f)
            final_data[file] = data
        except Exception as error:
            print(error)

final_data_json = json.dumps(final_data, sort_keys=True) #indent=2
with open(profileId + '.json', 'w') as outfile:
    outfile.write(final_data_json)  
    

FileNotFoundError: [Errno 2] No such file or directory: '/Users/michalpietrawski/Library/CloudStorage/OneDrive-Personal/GitHub/python-scripts/jupiter_notebook/Noble/'

In [ ]:
driver.quit()